<a href="https://colab.research.google.com/github/Gracey701/Project-2/blob/main/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, RocCurveDisplay

In [27]:
rice = pd.read_excel('/content/Rice_Cammeo_Osmancik.xlsx')
rice.info()
rice.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   int64  
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   int64  
 6   Extent             3810 non-null   float64
 7   Class              3810 non-null   object 
dtypes: float64(5), int64(2), object(1)
memory usage: 238.2+ KB


,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231,525.578979,229.749878,85.093788,0.928882,15617,0.572896,Cammeo
1,14656,494.311005,206.020065,91.730972,0.895405,15072,0.615436,Cammeo
2,14634,501.122009,214.106781,87.768288,0.912118,14954,0.693259,Cammeo
3,13176,458.342987,193.337387,87.448395,0.891861,13368,0.640669,Cammeo
4,14688,507.166992,211.743378,89.312454,0.906691,15262,0.646024,Cammeo


1. Source of data
- found out from uci (https://archive.ics.uci.edu/ml/datasets/Rice+%28Cammeo+and+Osmancik%29#)
- Didn't have the data so i found it on kaggle (https://www.kaggle.com/datasets/muratkokludataset/rice-dataset-commeo-and-osmancik)
2. Brief description of data
- Different features of rice with 2 different types of what the rice could be
3. What is the target?
- type of rice (cammeo or osmancik)
4. What does one row represent? (A person?  A business?  An event? A product?)
- a grain of rice
5. Is this a classification or regression problem?
- classification
6. How many features does the data have?
- 7 not including the target
7. How many rows are in the dataset?
- 3810
8. What, if any, challenges do you foresee in cleaning, exploring, or modeling with this dataset?
- exploring the data as the units and features seem foreign to understand

- /content/car.names


In [22]:
df = pd.read_csv('/content/car.data')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   vhigh    1727 non-null   object
 1   vhigh.1  1727 non-null   object
 2   2        1727 non-null   object
 3   2.1      1727 non-null   object
 4   small    1727 non-null   object
 5   low      1727 non-null   object
 6   unacc    1727 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


,vhigh,vhigh.1,2,2.1,small,low,unacc
0,vhigh,vhigh,2,2,small,med,unacc
1,vhigh,vhigh,2,2,small,high,unacc
2,vhigh,vhigh,2,2,med,low,unacc
3,vhigh,vhigh,2,2,med,med,unacc
4,vhigh,vhigh,2,2,med,high,unacc


In [25]:
#deleting last column as it's not listed as an attribute by the source and seems unnecessary 
df = df.iloc[:,:-1]

In [26]:
df.columns = ['buy rate','maint','doors','persons','lug_boot','safety']
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1727 entries, 0 to 1726
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buy rate  1727 non-null   object
 1   maint     1727 non-null   object
 2   doors     1727 non-null   object
 3   persons   1727 non-null   object
 4   lug_boot  1727 non-null   object
 5   safety    1727 non-null   object
dtypes: object(6)
memory usage: 81.1+ KB


,buy rate,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,med
1,vhigh,vhigh,2,2,small,high
2,vhigh,vhigh,2,2,med,low
3,vhigh,vhigh,2,2,med,med
4,vhigh,vhigh,2,2,med,high


1. Source of data
- from uci site
- https://archive.ics.uci.edu/ml/datasets/Car+Evaluation

2. Brief description of data
- Various features of cars that may potentially contribute to the safety of the car
3. What is the target?
- safety
4. What does one row represent? (A person?  A business?  An event? A product?)
- car
5. Is this a classification or regression problem?
- classification
6. How many features does the data have?
- 5, not including the target
7. How many rows are in the dataset?
- 1727
8. What, if any, challenges do you foresee in cleaning, exploring, or modeling with this dataset?
- working with 3 target classes instead of 2 (low, med, high)